In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('moka-ai/m3e-base')
sentences = ['示例句子1', '示例句子2']
embeddings = model.encode(sentences)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[-0.02429847, -0.14313667,  0.08467216, ..., -0.5409736 ,
        -0.3786679 , -0.35205388],
       [-0.15109941,  0.14337362,  0.39803982, ..., -0.55295366,
        -0.23078297, -0.38428977]], dtype=float32)

In [ ]:
print(embeddings.shape, embeddings.dtype)

(2, 768) float32


# 2. LangChain 方法

In [ ]:
!pip install -q langchain sentence_transformers chromadb
!pip install -q langchain-community
!pip install -qU langchain-openai
print('Successfully install all packages')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.5 MB/s eta 0:00:00
Successfully install all packages


In [ ]:
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


import os
from google.colab import files


print('Successfully import all dependency')

Successfully import all dependency


In [ ]:
# remember to set upi key manaually each time, os.environ["O#P#E#N#A#I#_#A#P#I#_#K#E#Y"] = ""
llm = OpenAI(temperature=0)

print('Successfully set O#P#E#N#I#_#A#P#I#_#K#E#Y and LLM')

In [ ]:
# 选择并上传文件
uploaded = files.upload()

# 列出上传的文件
for filename in uploaded.keys():
    print(f"已上传文件：{filename}")

Saving YuSunWiscAI.txt to YuSunWiscAI.txt
已上传文件：YuSunWiscAI.txt


In [ ]:
# 查看当前工作目录
%pwd

# 列出当前目录下的所有文件和文件夹
%ls

sample_data/  YuSunWiscAI.txt


In [ ]:
# Load the text data
loader = TextLoader('YuSunWiscAI.txt')

# Create an embedding model instance
embedding_model = OpenAIEmbeddings()

# Create a VectorstoreIndexCreator instance with the embedding model
index_creator = VectorstoreIndexCreator(embedding=embedding_model)

# Create an index from the loader
index = index_creator.from_loaders([loader])

# Create an OpenAI LLM instance
llm = OpenAI(temperature=0)

# Define the query
query = "What's the net worth change of YuSunWiscAI from 2020 to 2024?"

# Separator line
separator = '-' * 80

# Result 1: Using RAG
rag_result = index.query(query, llm=llm)
print("\nResult using RAG:\n")
print(rag_result)
print(f"\n{separator}\n")

# Result 2: Direct LLM query
llm_result = llm.invoke(query)
print("\nResult using LLM:\n")
print(llm_result)
print(f"\n{separator}\n")

# Result 3: LLM with detailed prompt
# Load the content of the text file
document_content = loader.load()

# Construct a detailed prompt
detailed_prompt = (
    f"Please read the following financial report and answer the question:\n\n"
    f"{document_content}\n\n"
    f"Question: {query}"
)

# Get the result using the detailed prompt
llm_prompt_result = llm.invoke(detailed_prompt)
print("\nResult using LLM with detailed prompt:\n")
print(llm_prompt_result)
print(f"\n{separator}\n")

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(



Result using RAG:

 The net worth change of YuSunWiscAI from 2020 to 2024 is an increase of $28,000,000.

--------------------------------------------------------------------------------


Result using LLM:



It is impossible to accurately predict the net worth change of YuSunWiscAI from 2020 to 2024 as it depends on various factors such as the success of their business ventures, investments, and market conditions. Additionally, the net worth of an AI entity may not follow the same patterns as that of a human individual.

--------------------------------------------------------------------------------


Result using LLM with detailed prompt:



The net worth change of YuSunWiscAI from 2020 to 2024 is an increase of $23,000,000.

--------------------------------------------------------------------------------

